In [ ]:
!pip3 install --quiet tensorflow-text

import csv
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text  # Imports TF ops for preprocessing.
import pandas as pd

def change_labels(x):
    return 1 if x == "spam" else 0

data = pd.read_csv(
    "./input/MatrixData.tsv", sep="\t", quoting=csv.QUOTE_NONE, encoding="utf-8"
)

# Remove unknown
data.dropna(inplace=True)
data["label"] = data["label"].apply(change_labels)

# Remove stopwords
#data["message"] = data["message"].apply(remove_stopwords)

# Shuffle data
data = data.sample(frac=1).reset_index(drop=True)

# Split data into messages and label sets
sentences = data["message"].tolist()
labels = data["label"].tolist()

# Separate out the sentences and labels into training and test sets
# training_size = int(len(sentences) * 0.8)
training_size = int(len(sentences) * 0.7)
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

# Make labels into numpy arrays for use with the network later
testing_labels_final = np.array(testing_labels)
training_labels_final = np.array(training_labels)
training_sentences_final = np.array(training_sentences)
testing_sentences_final = np.array(testing_sentences)

# Load the BERT encoder and preprocessing models
preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert = hub.load('https://tfhub.dev/google/electra_base/2')

# Convert the sentences to bert inputs
bert_inputs = preprocess(sentences)

# Feed the inputs to the model to get the pooled and sequence outputs
bert_outputs = bert(bert_inputs, training=False)
pooled_output = bert_outputs['pooled_output']
sequence_output = bert_outputs['sequence_output']

print('\nSentences:')
print(sentences)
print('\nPooled output:')
print(pooled_output)
print('\nSequence output:')
print(sequence_output)
